# Projeto Prático #4 
## Multilayer Perceptron + GridSearchCV + WheatSeedDataset

## Bibliotecas

In [59]:
import pandas as pd
import numpy as np
from math import ceil
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')

## Dataset

In [60]:
dataset = pd.read_csv('WheatSeedDataset.csv', sep='\t')
dataset.head()

,Area,Perimeter,Compactness,Length of Kernel,Width of Kernel,Asymmetry Coefficient,Length of Kernel Groove,Type
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1


## Regra da Pirâmide Geométrica

        Nh = α·√(Ni·No) ; Nh = Número de Neurônios Ocultos
                          Ni = Número de Neurônios de Entrada
                          No = Número de Neurônios de Saída
                          α  = [0.5, 2, 3]

In [3]:
def piramide_geometrica(ni, no, alfa):
    nh = alfa*((ni*no)**(1/2))
    return ceil(nh)

##  Distribuição dos Neurônios em duas Camadas Ocultas

Função para gerar todas as possíveis 2-tuplas que representam o número de neurônios distribuídos por duas camadas ocultas de uma RNA do tipo MLP, dado o número de neurônios ocultos obtidos previamente pela Regra da Pirâmide Geométrica.

In [4]:
def hidden_layers(layers, nh):
    for i in range(1, nh):
        neurons_layers = (i, nh-i)
        layers.append(neurons_layers)
    return layers

### Criação de Lista de Camadas Ocultas a Partir da Regra da Pirâmide Geométrica

In [5]:
num_in = 7
num_out = 3
alpha = [0.5, 2, 3]
layers = []

In [6]:
for i in range(len(alpha)):
    nh = piramide_geometrica(num_in, num_out, alpha[i])
    print('Para α = %.1f, Nh = %d'%(alpha[i],nh))
    hidden_layers(layers, nh)#insere cada possibilidade de camadas ocultas, dado o numero de neurônios, na lista 'layers'
    
print()
print('Distribuições de Camadas Ocultas:\n')
for i in layers:
    print(i)

Para α = 0.5, Nh = 3
Para α = 2.0, Nh = 10
Para α = 3.0, Nh = 14

Distribuições de Camadas Ocultas:

(1, 2)
(2, 1)
(1, 9)
(2, 8)
(3, 7)
(4, 6)
(5, 5)
(6, 4)
(7, 3)
(8, 2)
(9, 1)
(1, 13)
(2, 12)
(3, 11)
(4, 10)
(5, 9)
(6, 8)
(7, 7)
(8, 6)
(9, 5)
(10, 4)
(11, 3)
(12, 2)
(13, 1)


In [7]:
parameters = {'solver': ['lbfgs'], 
              'activation': ['identity', 'logistic', 'tanh', 'relu'],
              'hidden_layer_sizes': layers,
              'max_iter':[1000],
              'learning_rate': ['adaptive', 'constant']}

gs = GridSearchCV(MLPClassifier(), 
                  parameters, 
                  cv=3, 
                  scoring='accuracy')

In [8]:
x = dataset.drop(['Type'], axis = 1)
y = dataset.Type

In [9]:
gs.fit(x, y)

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'solver': ['lbfgs'], 'activation': ['identity', 'logistic', 'tanh', 'relu'], 'hidden_layer_sizes': [(1, 2), (2, 1), (1, 9), (2, 8), (3, 7), (4, 6), (5, 5), (6, 4), (7, 3), (8, 2), (9, 1), (1, 13), (2, 12), (3, 11), (4, 10), (5, 9), (6, 8), (7, 7), (8, 6), (9, 5), (10, 4), (11, 3), (12, 2), (13, 1)], 'max_iter': [1000], 'learning_rate': ['adaptive', 'constant']},
       pre_dispatch='2*n_jobs', refit=True

In [50]:
gs.best_params_

{'activation': 'identity',
 'hidden_layer_sizes': (6, 4),
 'learning_rate': 'adaptive',
 'max_iter': 1000,
 'solver': 'lbfgs'}

In [49]:
gs.best_score_ #melhor media acurácia para os testes

0.9333333333333333

In [51]:
for i in gs.cv_results_.keys():
    print (i)

mean_fit_time
std_fit_time
mean_score_time
std_score_time
param_activation
param_hidden_layer_sizes
param_learning_rate
param_max_iter
param_solver
params
split0_test_score
split1_test_score
split2_test_score
mean_test_score
std_test_score
rank_test_score
split0_train_score
split1_train_score
split2_train_score
mean_train_score
std_train_score


In [57]:
df = pd.DataFrame(gs.cv_results_)
df.head(5)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_activation,param_hidden_layer_sizes,param_learning_rate,param_max_iter,param_solver,params,...,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,0.333913,0.201537,0.001062,0.000160,identity,"(1, 2)",adaptive,1000,lbfgs,"{'activation': 'identity', 'hidden_layer_sizes...",...,0.942029,0.753623,0.861905,0.078813,74,0.862319,0.865248,0.914894,0.880820,0.024123
1,0.124696,0.074822,0.000985,0.000283,identity,"(1, 2)",constant,1000,lbfgs,"{'activation': 'identity', 'hidden_layer_sizes...",...,0.927536,0.753623,0.852381,0.072359,81,0.855072,0.836879,0.914894,0.868949,0.033326
2,0.123771,0.061240,0.000793,0.000043,identity,"(2, 1)",adaptive,1000,lbfgs,"{'activation': 'identity', 'hidden_layer_sizes...",...,0.913043,0.753623,0.842857,0.065948,91,0.891304,0.858156,0.914894,0.888118,0.023272
3,0.223674,0.123618,0.000814,0.000048,identity,"(2, 1)",constant,1000,lbfgs,"{'activation': 'identity', 'hidden_layer_sizes...",...,0.913043,0.753623,0.842857,0.065948,91,0.884058,0.851064,0.914894,0.883338,0.026063
4,0.226122,0.197053,0.001158,0.000493,identity,"(1, 9)",adaptive,1000,lbfgs,"{'activation': 'identity', 'hidden_layer_sizes...",...,0.942029,0.753623,0.852381,0.076625,81,0.891304,0.865248,0.914894,0.890482,0.020276
